In [1]:
import tensorflow as tf
import numpy as np
import math
tf.enable_eager_execution()

C:\Users\carso\Anaconda3\envs\tflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Gradient tapes

In [2]:
x = tf.ones((2, 2))
print("x:\n{}".format(x))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    print("y:\n{}".format(y))
    z = tf.multiply(y, y)
    print("z:\n{}".format(z))
    
# Derivative of z with respect to original input tensor x
dz_dx = t.gradient(z, x)
print("dz/dy:\n{}".format(dz_dx))
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

x:
[[1. 1.]
 [1. 1.]]
y:
4.0
z:
16.0
dz/dy:
[[8. 8.]
 [8. 8.]]


In [3]:
x = tf.ones((2, 2))
print("x:\n{}".format(x))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    print("y:\n{}".format(y))
    z = tf.multiply(y, y)
    print("z:\n{}".format(z))
    
# Use tape to compute the derivative of z with respect to the
# intermediate value y
dz_dy = t.gradient(z, y)
print("dz/dy:\n{}".format(dz_dy))
assert dz_dy.numpy() == 8.0

x:
[[1. 1.]
 [1. 1.]]
y:
4.0
z:
16.0
dz/dy:
8.0


In [4]:
# Creating a persistent gradient tape in order to perform multiple gradients

x = tf.constant(3.0)
print("x:\n{}".format(x))

with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    print("y: {}".format(y))
    z = y * y
    print("z: {}".format(z))
    
# 108.0 -> d/dx x^4 -> (4 * x^3 @ x = 3)
dz_dx = t.gradient(z, x)
print("dz/dx: {}".format(dz_dx))

# 6.0 -> d/dx x^2 -> (2 * x @ x = 3)
dy_dx = t.gradient(y, x)
print("dy/dx: {}".format(dy_dx))

x:
3.0
y: 9.0
z: 81.0
dz/dx: 108.0
dy/dx: 6.0


## Recording control flow

In [16]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
        print("out: {}".format(out))
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

out: 8.0
out: 8.0
out: 4.0


## Higher-order gradients

In [23]:
x = tf.contrib.eager.Variable(1.0)

with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        y = x * x * x
    # compute gradient inside the 't1' context manager
    # which means gradient computation is differentiable as well
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t1.gradient(dy_dx, x)
print("dy/dx: {}".format(dy_dx))
print("d2y/dx2: {}".format(d2y_dx2))

dy/dx: 3.0
d2y/dx2: 6.0
